In [2]:
import numpy as np
import pandas as pd
import glob

## Read

In [3]:
glob.glob('data/*')

['data\\clustering_1.txt',
 'data\\clustering_2.txt',
 'data\\clustering_3.txt',
 'data\\clustering_4.txt',
 'data\\clustering_5.txt',
 'data\\partitions.txt']

In [51]:
# read clustering and partitions data
clustering1 = pd.read_table('data/clustering_1.txt', sep=' ', header=None, index_col=0)
partitions = pd.read_table('data/partitions.txt', sep =' ', header=None, index_col=0)
clustering1.columns = ['cluster']
partitions.columns = ['partition']

# merge two df
data = partitions.merge(clustering1, left_index=True, right_index=True)
n = data.shape[0]

## Prob

In [76]:
# calculate pij for each cluster
pij = data.groupby(['cluster','partition']).size().to_frame('count')/n #.reset_index()

# the probability of getting cluter i: #ni/n
pci = data.groupby('cluster').count()/n

# the probability of getting partition j : #nj/n
ptj = data.groupby('partition').count()/n

In [124]:
pij.loc[1]*np.log(np.divide(pij.loc[1], np.multiply(ptj, pci.loc[1].values.reshape(1,1)) ))

,count
partition,
0,-0.011689
1,0.345369
2,-0.018756


,count
partition,
0,-3.506558
1,1.068153
2,-2.813411
